# Automated Reasoning Checks with Amazon Bedrock Guardrails Guide
## Introduction
This guide demonstrates how to implement and utilize Automated Reasoning (AR) checks with Amazon Bedrock Guardrails. AR policies provide a systematic way to validate model responses against business rules and automatically correct any policy violations, ensuring consistent compliance across your AI applications.

> **Note:** This feature is in gated preview and you only have access to it if you've been allow listed

## Contents
- Basic Setup and Configuration
- Loading Service Models
- Guardrail Creation and Management
- Input Data Formatting
- AR Policy Validation
- Response Correction and Feedback
- Sample Implementation

## Prerequisites
- An AWS account with Bedrock access
- Appropriate IAM roles and permissions
- Access to Amazon Bedrock Guardrails
- AR policy configuration privileges
- Basic understanding of Python and AWS SDK

## Use case Introduction
For this demonstration, we'll focus on a Leave of Absence, Paid (LoAP) policy implementation - a common HR use case that highlights the capabilities of Automated Reasoning Checks. The LoAP policy is particularly suitable for this demonstration because it contains clearly defined eligibility criteria, specific duration limits, and detailed procedural requirements that can be effectively translated into logical rules for validation. Implementing these rules in Amazon Bedrock Automated Reasoning Checks involves two stages: first creating the policy via the console to establish the validation framework, and then using the validation API through ApplyGuardrails to programmatically verify AI-generated responses against the established rules.

## Creating Automated Reasoning Policy in Amazon Bedrock Console

Before implementing AR checks in our code, we need to create an Automated Reasoning policy through the Amazon Bedrock console. Follow these steps to create your policy:

### Step 1: Navigate to AR Policy Section
Navigate to **Amazon Bedrock > Safeguards > Automated Reasoning (Preview)**

![AR Policy Navigation](images/ar-policy-nav.png)

### Step 2: Create New Policy 
Click "Create policy" and provide:
- Policy name (e.g., "LoAP_Eligibility_Policy")
- Policy description

![AR Policy Navigation](images/create-policy.png)

### Step 3: Upload Policy Document
- For this demo, use the sample document in the "data" folder
> **Important:** The source content has the following limitations:
- Cannot be modified after creation
- Must not exceed 1500 tokens (~2 pages)
- Has limitations on table sizes and image processing

### Step 4: Define Policy Intent
For the LoAP policy, use this intent description:

> Create a logical model of the Leave of Absence, Paid (LoAP) policy in this document. Employees will ask questions about:
>
> - Eligibility requirements
> - LoAP allowance and duration
> - Benefits during time off
> - Return to work procedures
>
> Example:
> 
> **Q:** I am a temporary contractor working in operations. Am I eligible for LOAP?  
> **A:** No, only full time employees are eligible for LoAP.
After adding the policy intent, click "Create".

### Step 5: Wait for Creation
- Creation takes a few minutes
> **Note:** Go grab a coffee while the policy is being created
- Rules and variables are generated automatically
- You can edit/add rules and variables after creation


### Step 6: Review Policy Details
Check the policy details section for:
- Policy version
- Intent description
- Build status
- Generated rules and variables

![AR Policy Navigation](images/version-details.png)

### Step 7: Policy Ready
Your AR policy is now ready to be used in code implementation

![AR Policy Navigation](images/policy-ready.png)

> **Note:** Expand policy to get the Policy ID. Make sure to save your policy ID - you'll need it for the code implementation in the next sections.
![AR Policy ID](images/policy_id.png)


## Setting Up the Environment

Upgrade the version of boto3

In [ ]:
#!pip install boto3 --upgrade

First, let's import required libraries and configure our environment parameters.
> **Note:** You will need to replace the following configuration parameters with yours - DEFAULT_GUARDRAIL_NAME, DEFAULT_AR_POLICY_VERSION, region, ar_policy, and model_id

In [ ]:
import boto3
import botocore
import json
from enum import Enum
import os
from validation_client import ValidatingConversationalClient

# Configuration parameters
DEFAULT_GUARDRAIL_NAME = "<YOUR_GUARDRAIL_NAME>"  # e.g., "my_policy_guardrail"
DEFAULT_AR_POLICY_VERSION = "1"

# AWS configuration
region = "us-west-2"  
ar_policy = "<YOUR_AR_POLICY_ID>"        # The ID from your created Automated Reasoning policy
model_id = "<YOUR_BEDROCK_MODEL_ID>"     # Your chosen Bedrock model ID e.g "anthropic.claude-3-haiku-20240307-v1:0"

## Loading Service Models

Before we can use Bedrock's AR capabilities, we need to load the required service models. 
> **Note:** Create a folder with the name "models" and place the model files received in it. 

> Replace the "BEDROCK_MODEL_FILE" and the "RUNTIME_MODEL_FILE" with the model file names (in json format) 

The following code handles the model loading process:

In [ ]:
def add_service_model(model_file, service_name):
    """
    Loads and configures service models for Bedrock services.
    
    Args:
        model_file (str): Name of the model file
        service_name (str): Service identifier ('bedrock' or 'bedrock-runtime')
    
    Returns:
        bool: True if successful, False otherwise
    """
    # Set appropriate version based on service. 
    # These are the versions of the bedrock and bedrock runtime models given to you by the Account team. You can replace them if the suffix is different. 
    version = '2023-04-20' if service_name == 'bedrock' else '2023-09-30'
    
    # Configure paths
    source = f"models/{model_file}"
    dest_dir = os.path.expanduser(f"~/.aws/models/{service_name}/{version}")
    dest_file = f"{dest_dir}/service-2.json"

    try:
        # Create directory and copy file
        os.makedirs(dest_dir, exist_ok=True)
        with open(source) as f:
            model = json.load(f)
        with open(dest_file, 'w') as f:
            json.dump(model, f, indent=2)
        print(f"✓ Added model for {service_name}")
        return True
    except Exception as e:
        print(f"✗ Error adding {service_name}: {e}")
        return False

def main():
    # Define model configurations
    # Replace the <BEDROCK_MODEL_FILE> and the <RUNTIME_MODEL_FILE> with the model file names (in json format) 
    models = {
        '<BEDROCK_MODEL_FILE>': 'bedrock',
        '<RUNTIME_MODEL_FILE>': 'bedrock-runtime'
    }
    
    # Load each model
    for model_file, service_name in models.items():
        add_service_model(model_file, service_name)

if __name__ == "__main__":
    main()

### Create the bedrock and bedrock-runtime clients using Boto3

In [ ]:
boto_session = boto3.Session(region_name=region)
runtime_client = boto_session.client("bedrock-runtime")
bedrock_client = boto_session.client("bedrock")

## Creating and Managing Guardrails

Next, we'll implement functions to find existing guardrails or create new ones with AR policies. This code handles both the search and creation process:

In [ ]:
def find_guardrail_id(client, name) -> tuple[str, str]:
    """
    Finds the ID and version of a guardrail by its name.
    
    Args:
        client: The Bedrock client object
        name (str): Name of the guardrail to find
    
    Returns:
        tuple[str, str]: Guardrail ID and version if found, None otherwise
    """
    next_token = None
    while True:
        # List existing guardrails
        resp = client.list_guardrails(
        ) if next_token is None else client.list_guardrail(nextToken=next_token)

        # Search for matching guardrail
        for g in resp["guardrails"]:
            if g["name"] == name:
                return g["id"], g["version"]

        # Handle pagination
        if "nextToken" in resp and resp["nextToken"] != "":
            next_token = resp["nextToken"]
        else:
            break
    return None, None

# Find or create guardrail with AR policy
try:
    # First, try to find existing guardrail
    guardrail_id, guardrail_version = find_guardrail_id(
        bedrock_client, DEFAULT_GUARDRAIL_NAME)
    
    # If not found, create new guardrail
    if guardrail_id is None:
        create_resp = bedrock_client.create_guardrail(
            name=DEFAULT_GUARDRAIL_NAME,
            description="Automated Reasoning checks demo guardrail",
            automatedReasoningPolicyConfig={
                "policyIdentifier": ar_policy,
                "policyVersion": DEFAULT_AR_POLICY_VERSION
            },
            blockedInputMessaging='Input is blocked',
            blockedOutputsMessaging='Output is blocked',
        )
        guardrail_id = create_resp["guardrailId"]
        guardrail_version = create_resp["version"]
        print(f"✓ Created new guardrail: {guardrail_id}")
    else:
        print(f"✓ Found existing guardrail: {guardrail_id}")
        
except botocore.exceptions.ClientError as e:
    print(f"✗ Error managing guardrail: {str(e)}")
    raise

## Initializing the Validation Client

The `ValidatingConversationalClient` is a custom client class that manages interactions between Bedrock models and AR policy validations. This client handles the complete workflow of:
- Managing model conversations
- Applying AR policy checks
- Processing validation feedback
- Implementing automated corrections
- Maintaining conversation history

In [ ]:
# Initialize the validation client with our configurations
conversation = ValidatingConversationalClient(
    bedrock_client=runtime_client,    # Bedrock runtime client for model interactions
    guardrail_id=guardrail_id,        # ID from our created/found guardrail
    guardrail_version=guardrail_version,  # Corresponding guardrail version
    model=model_id,                   # Bedrock model identifier
)

## Processing Questions with AR Policy Validation

Now we'll implement the main function that processes questions, validates answers against AR policies, and handles any necessary corrections:

In [ ]:
def process_qa(question, manual_answer=False, debug=False):
    """
    Process a question-answer pair with AR policy validation.
    
    Args:
        question (str): The question to process
        manual_answer (bool): Whether to use manual input for answers
        debug (bool): Whether to show detailed validation feedback
    """
    # Get answer (either manual or from model)
    if manual_answer:
        answer = input("Enter your answer: ")
        interaction = conversation.add_qa(question, answer)
    else:
        try:
            interaction = conversation.ask_question(question)
            print(f"Original Answer => {interaction.answer}")
        except botocore.exceptions.ClientError as e:
            print(f"✗ Question processing failed: {str(e)}")
            return

    # Validate against AR policy
    try:
        feedback = conversation.validate_interaction(interaction)
        
        # Handle debug output
        if debug:
            print("\nAutomated Reasoning Feedback:")
            print("-----------------------------")
            print(json.dumps(feedback.raw_findings, indent=4))
            print("-----------------------------")
        else:
            # Show validation status
            status = "❌ INVALID" if feedback.is_invalid() else "✓ VALID"
            print(f"\nValidation Result: {status}")
            
            # Display policy violations if any
            if feedback.is_invalid():
                print("\nPolicy Violations:")
                for rule in feedback.invalid_rules():
                    print(f"➤ {rule}")
                print("\nSuggested Corrections:")
                for suggestion in feedback.suggestions():
                    print(f"➤ {suggestion}")

        # Get corrected response if needed
        if feedback.is_invalid():
            interaction = conversation.rewrite_answer(interaction, feedback)
            print(f"\nCorrected Response => {interaction.rewritten_answer}")
            
    except botocore.exceptions.ClientError as e:
        print(f"✗ Validation failed: {str(e)}")

## Testing AR Policy Validation

Let's test our implementation with some sample questions. This will demonstrate how the AR policy validates responses and handles policy violations.

### Sample Test Case
**Question:**
I am a part-time employee, am I eligible for LoAP?


**Manual Answer:**
Yes, part time employees are allowed to use LoAP

Let's run this test using our validation system:

In [ ]:
# Interactive testing
question = input("Enter your question: ")
process_qa(question, manual_answer=True, debug=True)
# After running the process_qa example, copy and paste the question and manual answer in the markdown above to test your policy validation. 

## AR Policy Validation Output

When we run our test case through the AR policy validation, the output looks like this:
> **Note:** You may get a different output depending on your policy document, configuration, and QnA.



### Automated Reasoning Feedback
![](images/output.png)

After getting a response, you can use the feedback suggestions to improve your LLM’s responses by iterating over the rules and variables from any VALID with suggestions and/or INVALID results. These collected variables and rules can then be fed back to your LLM, prompting it to revise its original answer based on the Automated Reasoning checks' feedback, with the accuracy of this process dependent on well-refined variable descriptions in your policy.
